# Inferring the goals of autonomous agents in Gen.jl

what is Gen.jl (2 sentences)

In [1]:
using Gen
using Gadfly

## 1. Modeling an autonomous agent

Explain ( 2 sentence). Explain that a model is a probabilisttic program, and that @program is used to define a probabilistic program in Gen.jl

In [2]:
include("scene.jl")
include("path_planner.jl")

planner_demo (generic function with 1 method)

In [143]:
@program model() begin
    
    # assumed scene
    scene = Scene(0, 100, 0, 100) # the scene spans the square [0, 100] x [0, 100]
    add!(scene, Tree(Point(30, 20))) # place a tree at x=30, y=20
    add!(scene, Tree(Point(83, 80)))
    add!(scene, Tree(Point(80, 40)))
    
    # time points at which we observe the agent's location
    observation_times = collect(linspace(0.0, 200.0, 20)) ~ "times"
    
    # assumed speed of the agent
    speed = 1.0
    
    # the starting location of the agent is a random point in the scene
    start = Point(uniform(0, 100), uniform(0, 100)) ~ "start"
    
    # the destination of the agent is a random point in the scene
    destination = Point(uniform(0, 100), uniform(0, 100)) ~ "destination"
    
    # the path of the agent from its start location to its destination
    # uses a simple 2D holonomic path planner based on RRT (path_planner.jl)
    (tree, rough_path, final_path) = plan_path(start, destination, scene)
    
    if isnull(final_path)
        
        # the agent could not find a path to its destination
        # assume it stays at the start location indefinitely
        locations = [start for _ in observation_times]
    else
        
        # the agent found a path to its destination
        # assume it moves from the start to the destinatoin along the path at constnat speed
        # sample its location along this path for each time in observation times
        locations = walk_path(get(final_path), speed, observation_times)
    end
    
    # assume that the observed locations are noisy measurements of the true locations
    # assume the noise is normally distributed with standard deviation 'noise'
    noise = 1.0
    for (i, t) in enumerate(observation_times)
        measured_x = normal(locations[i].x, noise) ~ "x$i"
        measured_y = normal(locations[i].y, noise) ~ "y$i"
    end
end;

We can run the model to generate probable scenarios:. We run a model using `@generate`, which executes a program and populates a trace with the values of the named expressions in the program.

In [4]:
trace = Trace()
@generate(trace, model())
start = value(trace, "start")
destination = value(trace, "destination")
xs = map((i) -> value(trace, "x$i"), 1:4)
ys = map((i) -> value(trace, "y$i"), 1:4)
println("start: ", start)
println("destination: ", destination)
println("xs: ", xs)
println("ys: ", ys)

start: Point(71.606425761579,28.652998508320906)
destination: Point(78.82389894214725,45.32472252282054)
xs: [71.4544,71.5963,71.9881,72.0496]
ys: [28.2453,29.8432,26.3642,28.6138]


If we run it again, we get a different result:

In [5]:
trace = Trace()
@generate(trace, model())
start = value(trace, "start")
destination = value(trace, "destination")
xs = map((i) -> value(trace, "x$i"), 1:4)
ys = map((i) -> value(trace, "y$i"), 1:4)
println("start: ", start)
println("destination: ", destination)
println("xs: ", xs)
println("ys: ", ys)

start: Point(63.55071647582833,75.11185462354821)
destination: Point(13.563001132276064,90.38443337296431)
xs: [63.4356,61.9607,62.6379,63.0589]
ys: [73.9472,75.8092,75.0515,76.1863]


# 2. Visualizing the probabilistic behavior of a model using a trace rendering

Printing out the values of variables is not a very good way to understand the probabilistic behavior of a program. Instead, we use a **trace rendering** to produce a visual representation of the trace. The trace rendering encodes the trace into a representation that the human visual system can quickly interpret. Here is an example trace rendering for our model that uses matplotlib:

javascript"""
var x = $(rand());
console.log(x)
"""

In [6]:
macro javascript_str(s) display("text/javascript", s); end

@javascript_str (macro with 1 method)

In [236]:
javascript"""

    function find_choice(trace, name) {
        if (name in trace.recorded) {
            //console.log(name + " found in interventions");
            return { value: trace.recorded[name], where: "recorded" }
        } else if (name in trace.interventions) {
            //console.log(name + " found in interventions");
            return { value: trace.interventions[name], where: "interventions"}
        } else if (name in trace.constraints) {
            //console.log(name + " found in constraints");
            return { value: trace.constraints[name], where: "constraints"}
        } else {
            return null
        }
    }

    Jupyter.notebook.kernel.comm_manager.unregister_target("agent_model_renderer");
    Jupyter.notebook.kernel.comm_manager.register_target("agent_model_renderer", function(comm, msg) {
            comm.on_msg(function(msg) {

                var id = msg.content.data.svg_element_id;
                var s = Snap("#" + id);
                var trace = msg.content.data.trace;
                
                var radius = 3;
                var alpha = 0.2;
                                
                // render start
                var start = find_choice(trace, "start");
                if (start) {
                    var circle = s.select("#start");
                    if (!circle) {
                        circle = s.circle(start.value.x, start.value.y, radius);
                        circle.attr({id: "start", fill: "#BE6698", stroke: "#000",
                                     
                                     "fill-opacity": alpha});
                    }
                    circle.attr({cx: start.value.x, cy: start.value.y, 
                                 strokeWidth: start.where == "interventions" ? 4 : 1});
                }
                
                // render destination
                var destination = find_choice(trace, "destination");
                if (destination) {
                    var circle = s.select("#destination");
                    if (!circle) {y
                        circle = s.circle(destination.value.x, destination.value.y, radius);
                        circle.attr({id: "destination", fill: "#547F96", stroke: "#000", strokeWidth: 1, "fill-opacity": alpha});
                    }
                      circle.attr({cx: destination.value.x, cy: destination.value.y, 
                                 strokeWidth: destination.where == "interventions" ? 4 : 1});
                }
                
                // render points along the path
                var times = find_choice(trace, "times");
                var num_path_points = times.value.length;
                for (var i=1; i<=num_path_points; i++) {
                    var id = "path-" + i;
                    var circle = s.select("#" + id);
                    var x = find_choice(trace, "x" + i);
                    var y = find_choice(trace, "y" + i);
                    if (!circle) {
                        circle = s.circle(x.value, y.value, radius)
                        circle.attr({id: id, fill: "#EDD37F", stroke: "#000", "fill-opacity": alpha});
                    }
                    circle.attr({cx: x.value, cy: y.value,
                                 // interesting... can visualize interventions with lines...
                        strokeWidth: (x.where == "interventions" || y.where == "interventions") ? 4 : 1});

                }

                
            })});
"""

In [237]:
# TODO: define a renderer API in Gen (that is general enough to handle Unreal engine, and this thing..)
import IJulia
# NOTE: this is all generic. we can make this into a special thing...
type ModelSVGRenderer
    # TODO need to load that Rendering class from the .js file somehow into the JS context.
    rendering_id::String # The target name for Javascript
    svg_element_id::Nullable{String} # SVG DOM ID
    comm::IJulia.Comm # communication object to Javascript
    
    function ModelSVGRenderer()
        new(Nullable{String}())
    end
    
    function ModelSVGRenderer(svg_element_id::String)
        rendering_id = "agent_model_renderer"
        #js = """
        #"""
        #display(HTML(js))
        #sleep(10)
        comm = IJulia.Comm(rendering_id, data=Dict())
        # TODO do we have to wait for the JS to register it (it is asynchronous I think)
        new(rendering_id, Nullable{String}(svg_element_id), comm)
    end
end

function canvas(renderer::ModelSVGRenderer)
     # this should cause it to return the SVG tag (which will be put into the DOM) and send that tag to the JS.
    renderer.svg_element_id = string(Base.Random.uuid4())
    HTML("<svg id=$(renderer.svg_element_id)></svg>")
end

function render(renderer::ModelSVGRenderer, trace::Trace)
    if (isnull(renderer.svg_element_id))
        error("No canvas has been created yet")
    end
    IJulia.send_comm(renderer.comm, Dict(
            "trace" => trace,
            "svg_element_id" => renderer.svg_element_id))
end

render (generic function with 3 methods)

In [238]:
HTML("""<svg id="test" width=100 height=100></svg>""")

HTML{String}("<svg id=\"test\" width=100 height=100></svg>")

In [239]:
renderer = ModelSVGRenderer("test")

ModelSVGRenderer("agent_model_renderer",Nullable{String}("test"),IJulia.CommManager.Comm{:agent_model_renderer}("94e9e301-7a1b-4596-808f-e91b243c3d87",true,IJulia.CommManager.noop_callback,IJulia.CommManager.noop_callback))

In [244]:
trace = Trace()
intervene!(trace, "start", Point(10., 10.))
for i=1:100
@generate(trace, model())
#render(renderer, trace)
    IJulia.send_comm(renderer.comm, Dict(
            "trace" => trace,
            "svg_element_id" => renderer.svg_element_id))
end

In [241]:
js = "<script>alert('Hello World!');</script>"
display(HTML(js))

HTML{String}("<script>alert('Hello World!');</script>")

In [242]:
type MatplotlibRenderer
    start_color::String
    destination_color::String
    measurement_color::String
end

function render(renderer::MatplotlibRenderer, trace::Trace)
    # fill in the rendering here. This will be not so pretty matplotlib code.
    # TOOD use Luxor.jl?
end

render (generic function with 3 methods)

Let's use this rendering to visualize a run of the our program:

In [14]:
renderer = MatplotlibRenderer("blue", "green", "red")
renderer(renderer, trace)

LoadError: MethodError: objects of type MatplotlibRenderer are not callable

In [15]:
renderer = SVGRenderer();
create(renderer)


LoadError: UndefVarError: SVGRenderer not defined

Since the behavior is stochastic, we need to visualize many samples at once in a grid, to get a sense of the full distribution:

In [16]:
function render_grid(renderer, traces::Vector{Trace})
    # TODO
end

traces = [begin trace = Trace(); @generate(trace, model()); trace end for i=1:100]
render_grid(renderer, traces)

We can intervene and see what the traces look like when we fix the value of `start` and `destination` to specific values:

In [17]:
trace = Trace()
intervene!(trace, "start", Point(0.5, 0.6))
intervene!(trace, "destinatoin", Point(0.7, 0.98))
traces = [begin trace = deepcopy(trace); @generate(trace, model()); trace end for i=1:100]
render_grid(renderer, traces)

LoadError: UndefVarError: trace not defined

In [18]:
# TODO use a JavaScript visualization
HTML("""<script>

    </script>""")

HTML{String}("<script>\n\n</script>")

# 3. Probabilistic inference

So far, we have simulated forward from the model, and we have intervened on some random choices and simulated the consequence. However, suppose we had observed a given sequence of locations of the agent, and we wanted to know probable goal locations? This is a query that cannot be answered simply by forward simulation of the program, because the location of the drone is a *consequence* and not a *cause* of the destination. We can easily find probable consequences given the causes, but finding probable causes given the consequences requires a bit more work.

Here is an example dataset showing measured locations for the first 15 time points:

In [19]:
observed_x = [0.1, 0.3, 0.5] # TODO..
observed_y = [0.1, 0.3, 0.5] # TODO

3-element Array{Float64,1}:
 0.1
 0.3
 0.5

The first step in inference is to constrain the random choices that are observed.

In [20]:
trace = Trace()
for i=1:15
    constrain!(trace, "x$i", observed_x[i])
    constrain!(trace, "y$i", observed_y[i])
end

LoadError: BoundsError: attempt to access 3-element Array{Float64,1} at index [4]

Now, when we run the program in this trace, we find that the score of the trace tells us how well the trace matches the constraints.

In [21]:
render_grid(renderer, traces)
# TODO render grid should show the score on top of the plot

We can use this score to filter out traces that don't match well with the observations. Specifically, we can sample a large number of traces, and pick one in proportion to the exponential of its score. We can then repreat this whole process a number of times:

In [22]:
# show SIR results in a tile plots

# 4. Improving the model

Our model above made a lot of assumptions that are unlikely to hold in the real world. For example, the agent always takes pretty direct paths from its starting location to its final destination. What if the agent is more unpredictable? What if it takes detours?

Here is a dataset that does not match our model's expectations. Let's see what happens when we try to do probabilistic inference in our model, given this data.

In [23]:
# show results (they should look bad)

This is an example of *model mis-specification*, which is when the model is not a good match for the distribution of data we are analyzing.

We can improve the model by adding the possibliltiy thathte agent takes a detour. Specifically, we add the possibility that the agent uses a waypoint and first walks from the starting locatoin to the waypoint and then from the waypoit to the final destination. Let's visualize the resulting samples:

In [24]:
# render_grid()...

Now, we can run inference as before:

In [25]:
# Do SIR with the new dataset and the improved model, and the same number of particles we used above.

In [26]:
# results for the same number of particles as above should look bad.

It looks like the results are not accurate. This is because in the new model, a random forward execution of the imporved model is a lot less likely to match the observations than a random forward exection of the original program. We can try to increase the number of samples to incrase the probability that we get one that matches the data. Note that this will take a few minutes t orun:

In [27]:
# Show results with a larger number of particles (should take < 1 min, should look okay)

This modification of the model mad einference a lot more computationally challenging, and our importance sampling algorihtm is not able to give us real-time inferneces. This motivates the need for a more sophisicated approach to probabilisitc inference.

# 5. Compiling inference with neural networks

There are a number of approaches for creating more efficient inference algorithms. We will focus on one approach, where we train a neural network to make informed guesses about he locatio nof the waypoint. First, let's understand in a bit more detail why the default inference algorihtm was slow

Suppose we knew the right waypoint:

In [28]:
trace = Trace()
constrain!(trace, "use-waypoint", true)
constrain!(trace, "waypoint", Point(0.5, 0.5))

Point(0.5,0.5)

Then the baseline importance sapmling algorithm gives reasonable inferences with fewer samples. We use this idea by training a neural network to make informed guesses about the waypoint, given the observed data as its input. We train the neural network on nany simulatoins of the program. Then, the resulting trained neural network can be used to make informed guesses about the waypooint given any observations.

In [29]:
# show the neural network, and show the training.

Let's visualize the guesses made by the neural network for a few  different datasets:

In [30]:
# show four renderings left to right of different datasets, with circles denoting the neural network's guess about the waypoint. 

Now, let's use this trained neural network to speed up inference:

In [31]:
# show the modififed SIR algorithm, using propose!

In [32]:
# show results for fewer particles, which should be noticeably faster than without the neural network.
# make an explicit comparison.